# ECSE 415 Intro to Computer Vision

Assignment 3_1

Author: Mustafa Javed

Images provided by Prof James J. Clark

Credit: Part of code has been taken from ECSE 415 Tutorials and Assignment.

In [ ]:
#import libraries
from sklearn import svm
import matplotlib.pyplot as plt
import numpy as np
import cv2
from skimage.feature import hog 
from sklearn.ensemble import RandomForestClassifier

#mount drive
from google.colab import drive
drive.mount('/content/drive')

#define dataset path. Edit this path according to the folder where dataset flower_subset is stored
path = '/content/drive/My Drive/ECSE 415/Assignment 3/'

Mounted at /content/drive


# 1 Image Classifcation using RF and SVM

In [ ]:
# load dataset
train_images = np.load(path + 'flower_subset.npz')['train_images']
train_labels = np.load(path + 'flower_subset.npz')['train_labels']
test_images = np.load(path + 'flower_subset.npz')['test_images']
test_labels = np.load(path + 'flower_subset.npz')['test_labels']

# Resize the train/test images to 64 x 64
images = []
for i in range(len(train_images)):
  images.append( cv2.resize( train_images[i], (64,64)) )
train_images = np.array(images)

images = []
for i in range(len(test_images)):
  images.append( cv2.resize( test_images[i], (64,64)) )
test_images = np.array(images)

print(test_images.shape)

[[0.21221469 0.22623889 0.2076483  ... 0.63913    0.62145757 0.56879165]
 [0.25323558 0.26062826 0.28423043 ... 0.64410424 0.6495615  0.62533366]
 [0.31837804 0.3141458  0.37962335 ... 0.61238463 0.64118892 0.64752748]
 ...
 [0.32625278 0.321358   0.25491489 ... 0.19796885 0.2380261  0.22388366]
 [0.30048924 0.2023787  0.18634057 ... 0.17588848 0.20166159 0.24494868]
 [0.26778828 0.24327608 0.23951488 ... 0.1814185  0.18990838 0.19955576]]
(90, 64, 64)


In [ ]:
def hog(images, image_size, cell_size, block_size, bins ):
  img_size = (image_size[0], image_size[1]) # h x w in pixels
  cell_size = (cell_size[0], cell_size[1])  # h x w in pixels
  block_size = (block_size[0], block_size[1])  # h x w in cells
  nbins = bins  # number of orientation bins

  # create HoG Object
  # winSize is the size of the image cropped to multiple of the cell size
  hog = cv2.HOGDescriptor(_winSize=(img_size[1] // cell_size[1] * cell_size[1],
                                  img_size[0] // cell_size[0] * cell_size[0]),
                        _blockSize=(block_size[1] * cell_size[1],
                                    block_size[0] * cell_size[0]),
                        _blockStride=(cell_size[1], cell_size[0]),
                        _cellSize=(cell_size[1], cell_size[0]),
                        _nbins=nbins)
  features = []
  for i in range(images.shape[0]):
    features.append(hog.compute((images[i]*255).astype(np.uint8)).reshape(1, -1)) #multiply the image with 255 as images are in float [0,1]
  
  features = np.vstack(features)   # stack rows of hog features to from array            
  return features

#compute the HoG features of train and test set
train_features = hog(train_images, (64,64), (8,8), (4,4) ,4)
test_features = hog(test_images, (64,64), (8,8), (4,4) ,4)

#print(train_features.shape)
#print(test_features.shape)

In [ ]:
# Fit a non-linear SVM classifier(use RBF kernel with gamma=`auto' and C=1)

clf = svm.SVC(gamma='auto', C=1.0, kernel = 'rbf')
clf.fit(train_features, train_labels)

#Predict labels of the test images by feeding the test features to the trained
#classifier and calculate classification accuracy.

predict_labels = clf.predict(test_features)

print(" acctual label : " + str(test_labels))
print(" Predicted labels: " + str(predict_labels))


matches = np.array([predict_labels == test_labels])
match_true= np.zeros_like(matches)
print( 100*match_true[matches == True].shape[0] /matches.shape[1])

accuracy = 100*clf.score(test_features,test_labels)
print("The accuracy of the SVC is: " + "{:4.2f}".format(accuracy) + "%")

 acctual label : [0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3
 3 3 3 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6 6 6 7 7 7 7
 7 7 7 7 7 7 8 8 8 8 8 8 8 8 8 8]
 Predicted labels: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
11.11111111111111
The accuracy of the SVC is: 11.11%


### Tune values of hyperparameters 'gamma' and 'C' to achieve test accuracy greater than 50%.

In [ ]:
# Fit a non-linear SVM classifier

clf = svm.SVC(gamma= 0.05, C=100.0, kernel = 'rbf')
clf.fit(train_features, train_labels)

predict_labels = clf.predict(test_features)

print(" acctual label : " + str(test_labels))
print(" Predicted labels: " + str(predict_labels))

accuracy = 100*clf.score(test_features,test_labels)
print("The accuracy of the SVC is: " + "{:4.2f}".format(accuracy) + "%")

 acctual label : [0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3
 3 3 3 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6 6 6 7 7 7 7
 7 7 7 7 7 7 8 8 8 8 8 8 8 8 8 8]
 Predicted labels: [0 0 0 0 0 3 0 0 0 0 0 7 5 1 1 1 1 6 1 1 1 2 3 2 2 2 2 2 2 6 5 3 1 3 3 3 3
 8 2 6 4 4 4 4 4 4 4 4 4 4 0 2 5 6 2 4 5 1 4 5 1 6 1 7 6 6 6 1 3 6 1 7 7 1
 7 3 1 7 7 0 0 8 8 8 8 0 0 8 0 3]
The accuracy of the SVC is: 61.11%


Fit a Random Forest(RF) classifier (set n estimators=10, max depth=5 and criterion='entropy') on the features and the class labels of the training images.

Predict labels of the test images by feeding the test features to the trained classifier and calculate classification accuracy.

In [ ]:
#Fit a Random Forest(RF) classifier (set n_estimators=10, max depth=5 and criterion=`entropy')
#Need to keep the random_state to a particular value to get same result in each run

clf = RandomForestClassifier(n_estimators=10, max_depth=5, criterion='entropy', random_state = 0)
clf.fit(train_features, train_labels)

predict_labels = clf.predict(test_features)
print(" acctual label : " + str(test_labels))
print(" Predicted labels: " + str(predict_labels))

accuracy = 100*clf.score(test_features,test_labels)
print("The accuracy of the SVC is: " + "{:4.2f}".format(accuracy) + "%")

 acctual label : [0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3
 3 3 3 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6 6 6 7 7 7 7
 7 7 7 7 7 7 8 8 8 8 8 8 8 8 8 8]
 Predicted labels: [4 4 0 4 4 1 8 7 0 4 4 4 1 0 4 0 4 6 1 1 4 2 2 2 2 1 2 2 2 1 1 3 1 5 1 2 2
 1 2 6 4 4 4 4 4 4 1 4 4 4 4 2 5 5 5 1 6 4 0 4 6 1 1 1 2 3 2 1 6 6 4 6 7 0
 4 1 1 4 0 0 4 8 4 1 1 4 1 8 1 2]
The accuracy of the SVC is: 34.44%


Tune values of hyperparameters `n estimators' and `max depth' to achieve
test accuracy greater than 50%.

In [ ]:
#Fit a Random Forest(RF) classifier, Need to keep the random_state to a particular value to get same result in each run

clf = RandomForestClassifier(n_estimators=100, max_depth=10, criterion='entropy', random_state = 0)
clf.fit(train_features, train_labels)

predict_labels = clf.predict(test_features)
print(" acctual label : " + str(test_labels))
print(" Predicted labels: " + str(predict_labels))

accuracy = 100*clf.score(test_features,test_labels)
print("The accuracy of the SVC is: " + "{:4.2f}".format(accuracy) + "%")

 acctual label : [0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3
 3 3 3 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6 6 6 7 7 7 7
 7 7 7 7 7 7 8 8 8 8 8 8 8 8 8 8]
 Predicted labels: [0 0 0 0 4 3 8 0 0 0 0 1 1 1 1 1 4 1 1 1 4 2 2 2 2 3 2 2 2 1 7 3 1 3 1 2 3
 1 3 6 4 4 4 4 4 4 1 4 4 4 4 6 5 2 1 1 5 1 1 5 6 1 6 7 2 6 6 1 6 6 1 1 0 0
 7 7 1 7 4 4 0 8 8 1 8 8 1 8 0 5]
The accuracy of the SVC is: 57.78%


Compare results of SVM and RF classifiers. Which one provides better
results? Experiment training both classifiers with a range of random states (try different random values for the argument `random state') and measure classification accuracy of the test set. Which classifier is more stable or robust to the change in random state?

In [ ]:
import random

# try different values of random state
for i in range(0,5):

  state = random.randint(0,100)
  clf = svm.SVC(gamma= 0.05, C=100.0, kernel = 'rbf' , random_state=state)
  clf.fit(train_features, train_labels)
  print( "Accurancy for SVM with random state = " + str(state) + " is " + str( clf.score(test_features,test_labels)) )


for i in range(0,5):
  state = random.randint(0,100)
  clf = RandomForestClassifier(n_estimators=100, max_depth=10, criterion='entropy', random_state = state)
  clf.fit(train_features, train_labels)
  print( "Accurancy for RF with random state = " + str(state) + " is " + str(clf.score(test_features,test_labels)))

Accurancy for SVM with random state = 40 is 0.6111111111111112
Accurancy for SVM with random state = 54 is 0.6111111111111112
Accurancy for SVM with random state = 72 is 0.6111111111111112
Accurancy for SVM with random state = 97 is 0.6111111111111112
Accurancy for SVM with random state = 60 is 0.6111111111111112
Accurancy for RF with random state = 13 is 0.5555555555555556
Accurancy for RF with random state = 57 is 0.5111111111111111
Accurancy for RF with random state = 45 is 0.5333333333333333
Accurancy for RF with random state = 74 is 0.4888888888888889
Accurancy for RF with random state = 31 is 0.4888888888888889


Using the hyperparameters set above, we get a better result (higher accuracy) using the SVM as compared to the RF classifier. 

Under different values of random state, SVM was more robust and stable to changes in random state.